Створення MLP классу

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Клас MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_sizes, activations):
        super(MLP, self).__init__()
        layers = []

        # Додавання першого прихованого шару
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(self.get_activation(activations[0]))

        # Додавання наступних прихованих шарів
        for i in range(1, hidden_layers):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(self.get_activation(activations[i]))

        # Додавання вихідного шару
        layers.append(nn.Linear(hidden_sizes[-1], 3))  # Кількість класів у Iris = 3

        self.model = nn.Sequential(*layers)

    def get_activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'sigmoid':
            return nn.Sigmoid()
        elif activation == 'tanh':
            return nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation: {activation}")

    def forward(self, x):
        return self.model(x)

# Завантаження і підготовка даних
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Завантаження даних Iris
iris = load_iris()
X = iris.data
y = iris.target

# Стандартизація даних
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Розбиття даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення датасетів та даталоадерів
train_dataset = IrisDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = IrisDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Конфігурація моделі
input_size = X_train.shape[1]
hidden_layers = 2
hidden_sizes = [16, 8]
activations = ['relu', 'relu']

model = MLP(input_size, hidden_layers, hidden_sizes, activations)

# Визначення критерію втрат та оптимізатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Навчальний цикл
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Оцінка моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Epoch 1/50, Loss: 1.1128883957862854
Epoch 2/50, Loss: 1.0883010998368263
Epoch 3/50, Loss: 1.077210083603859
Epoch 4/50, Loss: 1.06691125780344
Epoch 5/50, Loss: 1.0466227903962135
Epoch 6/50, Loss: 1.0215302929282188
Epoch 7/50, Loss: 1.001501977443695
Epoch 8/50, Loss: 0.9780186116695404
Epoch 9/50, Loss: 0.950746014714241
Epoch 10/50, Loss: 0.9204811751842499
Epoch 11/50, Loss: 0.8873806521296501
Epoch 12/50, Loss: 0.8549941182136536
Epoch 13/50, Loss: 0.8230581730604172
Epoch 14/50, Loss: 0.7845658510923386
Epoch 15/50, Loss: 0.7479056715965271
Epoch 16/50, Loss: 0.7132771834731102
Epoch 17/50, Loss: 0.670254684984684
Epoch 18/50, Loss: 0.6333749368786812
Epoch 19/50, Loss: 0.5976696535944939
Epoch 20/50, Loss: 0.5662562921643257
Epoch 21/50, Loss: 0.5374281890690327
Epoch 22/50, Loss: 0.509211353957653
Epoch 23/50, Loss: 0.49664201959967613
Epoch 24/50, Loss: 0.46755366027355194
Epoch 25/50, Loss: 0.4591039791703224
Epoch 26/50, Loss: 0.4372706413269043
Epoch 27/50, Loss: 0.42760